In [ ]:
!pip install tensorflow

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report

# Step 1: Load the Dataset
newsgroups = fetch_20newsgroups(subset='all')

# Step 2: Feature Extraction (TF-IDF Vectorization)
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.5)
X = vectorizer.fit_transform(newsgroups.data)
y = newsgroups.target

# Convert sparse matrix to dense matrix for TensorFlow
X = X.toarray()

# Step 3: Split the Data into Training, Validation, and Test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, random_state=42)

# Step 4: Build the Neural Network using TensorFlow/Keras
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(np.unique(y)), activation='softmax')  # Output layer for multi-class classification
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Step 5: Train the Model (will use GPU if available)
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=64)

# Step 6: Evaluate on the Test Set
test_loss, test_acc = model.evaluate(X_test, y_test)
y_test_pred = np.argmax(model.predict(X_test), axis=1)

# Print the results
print("\nTest Accuracy:", test_acc)
print("\nTest Classification Report:\n", classification_report(y_test, y_test_pred))
